In [22]:
import os
import numpy as np
import torch
from torch.optim import Adam

from tianshou.env import PettingZooEnv
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.policy import PPOPolicy
from tianshou.trainer import OnpolicyTrainer
from tianshou.utils.net.common import Net

from pettingzoo.utils import parallel_to_aec
from env.football_env import raw_env, FootballMAEnv

from tianshou.env import DummyVectorEnv


In [23]:
# Load PettingZoo env
tianshou_env = PettingZooEnv(raw_env())

ValueError: too many values to unpack (expected 2)

In [24]:
def get_single_policy():
    observation_shape = tianshou_env.observation_space.shape or tianshou_env.observation_space["player_1"].shape
    action_shape = tianshou_env.action_space.n or tianshou_env.action_space["player_1"].n

    net = Net(
        state_shape=observation_shape,
        action_shape=action_shape,
        hidden_sizes=[128, 128],
        device="cpu"
    )
    optim = Adam(net.parameters(), lr=1e-3)
    policy = PPOPolicy(
        net,
        optim,
        discount_factor=0.99,
        gae_lambda=0.95,
        max_grad_norm=0.5,
        vf_coef=0.5,
        ent_coef=0.01,
        reward_normalization=True,
        action_space=tianshou_env.action_space["player_1"]
    )
    return policy

In [26]:
train_envs = DummyVectorEnv([lambda: PettingZooEnv(raw_env()) for _ in range(8)])
test_envs = DummyVectorEnv([lambda: PettingZooEnv(raw_env()) for _ in range(4)])

policy = get_single_policy()

train_collector = Collector(policy, train_envs, VectorReplayBuffer(20000, len(train_envs)))
test_collector = Collector(policy, test_envs)

result = OnpolicyTrainer(
    policy,
    train_collector,
    test_collector,
    max_epoch=10,
    step_per_epoch=10000,
    repeat_per_collect=4,
    episode_per_test=4,
    batch_size=64
)

os.makedirs("models", exist_ok=True)
torch.save(policy.state_dict(), "models/ppo_tianshou_policy.pth")

ValueError: too many values to unpack (expected 2)